In [ ]:
# Внимание!!! Важно, что бы файлы с данными и исполняемый файл находились в одной папке, 
# тогда пути к тестовым и тренировочным наборам будут содержать только имена файлов.
# 
# В пути к тренировочным и тестовым данным запрежается использовать абсалютную адресацию, 
# то есть адресацию, в которой присутствуют имена папок. Путь должен содержать только имя файла.
#
# Напоминание: под моделью машинного обучения понимаются все действия с исходными данными, 
# которые необходимо произвести, что бы сопоставить признаки целевому значению.
#
#
# ПОДБОР ПАРАМЕТРОВ В ОТДЕЛЬНЫХ ФАЙЛАХ, важно там сохранять ramdom_seed для повторимости результата
# данные файлы обязательны к размещению в репозитории
##
#

### Подключение библиотек

In [ ]:
# данный блок предназначен только для подключения необходимых библиотек
# запрещается подключать библиотеки в других блоках
#
# установка дополнительных библиотек размещается прямо здесь (обязательно закоментированы)
#
# pip install
#pip install category_encoders

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split , GridSearchCV , cross_val_score , RandomizedSearchCV,KFold,RepeatedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder , LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import get_scorer
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import learning_curve
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_percentage_error
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import shap
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.feature_selection  import RFE
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PowerTransformer
from matplotlib.colors import ListedColormap
from sklearn.model_selection import cross_val_score
from sklearn import svm 
from sklearn.svm import LinearSVC
from itertools import product
from sklearn.inspection import DecisionBoundaryDisplay
from sklearn.inspection import permutation_importance
from IPython.display import display
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import roc_auc_score
import math

### Обучение и выполнение лучшей модели

In [ ]:
# В области находится одна, единственная, итоговая модель машинного обучения с однозначными, 
# зафиксированными параметрами
#
# В данной области категорически запрещается искать, выбирать, улучшать, оптимизировать, 
# тюниговать и т.д. модель машинного обучения

In [ ]:
# Путь к тренировочному набору
path_train = 'train.csv' # содержит только имя файла, без имен папок
df = pd.read_csv(path_train)
display(df.head())
# Путь к тестовому набору
path_test  = 'test.csv' # содержит только имя файла, без имен папок
df1 = pd.read_csv(path_test)
display(df1.head())

## Обработка train

In [ ]:
#все манипуляции есть подробно в файлах Block 2, Model, Features, поэтому тут я все сделаю в одном блоке
# чтобы не размножать много ячеек

In [ ]:
df = df.drop(columns=['ID'])
# Функции для обработки колонок с буквами, а также для их разделения
def volume(s):
    if len(s) <= 3:
        a = s
    else:
        a = s[:3]
    return a;

def turbo(df):
    b = []
    for i in range(len(df)):
        if df['Engine_volume'][i][4:] == 'Turbo':
            b.append(1)    
        else:
            b.append(0)
    return pd.Series(b);
# Делаю новую колонку Турбо, чтобы выделять машины с турбированным двигателем
# а также разделяю пробег от километров и делаю из этого целые числа
df['Turbo'] = 0
df['Turbo'] = turbo(df)
df['Engine_volume'] = [volume(i) for i in df['Engine_volume']]
df['Engine_volume'] = df['Engine_volume'].astype(float)
df.Mileage = [int(''.join(filter(str.isdigit, i ))) for i in df.Mileage]

# Ниже буду выбрасывать выброс, которые нашел по графикам
ind = df[df['Engine_volume'] == 0].index
# это некорректные данные, поэтому удаляем их
df = df.drop(index=ind)
inde = df[df['Engine_volume'] == 20].index
df = df.drop(index=inde)
df.Mileage.nlargest(15)
# если загуглить, то можно увидеть, что можно прочитать
# Книги рекордов Гиннеса цифра – 3,2 миллиона миль (5 149 900 км) – датируется маем 2013 года.
# логично, что все пробеги ниже этого очень большого числа подходят под условия
# убираем это большьшое число(удаляем по индексу)
df = df.drop(index = 3169)
# 0 пробег у авто 2010-х годов это очень странно, что на них вообще нет пробега
ind = df[df['Mileage'] == 0].index
df = df.drop(index=ind)
#display(df[df.duplicated()])
# есть дубликаты, поэтому убираем дубликаты
#display(df[df.duplicated()].head(60))
df_duplicates = df.drop('Price', axis = 1)
dupl_rows = df_duplicates[df_duplicates.duplicated()].index
df = df.drop(index=dupl_rows)
df = df.drop_duplicates(keep = 'first')
# удаляем модель, тк эта переменная нам больше не нужна, есть бренд
df = df.drop(columns=['Model'])
#display(df[df['Year_of_production'] < 1999].Price)
# Машина за почти за 50000 1995 года будет довольно сильно колебать мое обучение, поэтому считаю выбросом 
# и выбрасываю ее
df = df.drop(index=7146)
df = df[df['Year_of_production'] > 1965]

# убираю неинформативные переменных, которые отобрал в файле Features
df = df.drop(columns=['Doors','Cylinders','Leather_interior'])
# надо убрать ненужные преобразования в трансформере и оформить все к test тоже
display(df.head(3))

In [ ]:
# я это расписывал в файле Block2 и features, что сделаю заранее преобразование столбцов (пробег и налоги)
# возьму преобразование Бокса-Кокса для пробег и налога
# преобраазуем переменные пробег и налог
power = PowerTransformer(method='box-cox') 
minmax = MinMaxScaler()

mile = power.fit_transform(minmax.fit_transform(pd.DataFrame(df.Mileage)) + 1)
tax = power.fit_transform(minmax.fit_transform(pd.DataFrame(df.Tax)) + 1)


df.Mileage = mile
df.Tax = tax
#преобразовываю данные и записываю в колонки моего Х после трансформера
# скорее всего это преобразование буду делать до трансформера,то есть сразу преобразовывать фрейм по этим колонкам
# тк трудно будет сделать преобразования после pipeline

In [ ]:
cat_features = ['Brand','Type','Color']
num_features = ['Year_of_production','Mileage','Tax']
y = np.array(df.Price)
X = df.drop(columns=['Price'])
columns = ['Brand','Type','Color','Fuel_type','Gear_box','Drive_wheels','Year_of_production','Mileage','Tax','Engine_volume','Airbags','Turbo']

In [ ]:
fuel_type_map = [{
    'col':'Fuel_type', 
    'mapping':{'Diesel':3,'Petrol':2,'Hybrid':1,'LPG':0}
    }]
gear_box_map = [{
    'col':'Gear_box',
    'mapping':{'Tiptronic':3,'Automatic':2,'Variator':1,'Manual':0}
    }]
drive_wheels_map = [{
    'col':'Drive_wheels',
    'mapping':{'4x4':2,'Rear':1,'Front':0}
    }]

fuel_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = fuel_type_map))])

gear_box_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = gear_box_map))])

drive_wheels_transformer = Pipeline(steps=[
    ('ce',ce.OrdinalEncoder(mapping = drive_wheels_map))])

   
# для деревьев можно не стандартизировать данные в фичах 
CT = ColumnTransformer(transformers=[('cat',OrdinalEncoder(),cat_features),
                                     ('fuel_type', fuel_transformer, ['Fuel_type']),
                                     ('gear_box', gear_box_transformer, ['Gear_box']),
                                     ('drive_wheels', drive_wheels_transformer, ['Drive_wheels']),
                                     ('num','passthrough',num_features)],
                                      remainder = 'passthrough')

display(CT)

## Обработка test

In [ ]:
#все манипуляции есть подробно в файле Test_processing поэтому тут я все сделаю в одном блоке
# чтобы не размножать много ячеек

In [ ]:
df1 = df1.drop(columns=['ID'])
# Функции для обработки колонок с буквами, а также для их разделения
def volume(s):
    if len(s) <= 3:
        a = s
    else:
        a = s[:3]
    return a;

def turbo(df):
    b = []
    for i in range(len(df)):
        if df['Engine_volume'][i][4:] == 'Turbo':
            b.append(1)    
        else:
            b.append(0)
    return pd.Series(b);
# Делаю новую колонку Турбо, чтобы выделять машины с турбированным двигателем
# а также разделяю пробег от километров и делаю из этого целые числа
df1['Turbo'] = 0
df1['Turbo'] = turbo(df1)
df1['Engine_volume'] = [volume(i) for i in df1['Engine_volume']]
df1['Engine_volume'] = df1['Engine_volume'].astype(float)
df1.Mileage = [int(''.join(filter(str.isdigit, i ))) for i in df1.Mileage]
# Посмотрим теперь на 0 и 20л двигатели
#display(df1[df1['Engine_volume'] == 0])
# Объем двигателя Тойота Аква составляет 1.5 л. Мощность двигателей Toyota Aqua от 74 до 91 л.
#
#Автомобиль комплектовался моторами объёмом: 1.6, 2.0, 2.4 л.
#display(df1[df1['Engine_volume'] == 20])
# То есть это тоже неккоретные данные,которые мы можем убрать
ind1 = df1[df1['Engine_volume'] == 0].index
df1 = df1.drop(index=ind1)
inde1 = df1[df1['Engine_volume'] == 20].index
df1 = df1.drop(index=inde1)
# теперь посмотрим на дубликаты и уберем их, тк есть полностью одинаковые машины, но по цене разной - странно
# цена разная с дубликатами - находил в траин
df1_duplicates = df1.copy()
dupl_rows1 = df1_duplicates[df1_duplicates.duplicated()].index
df1 = df1.drop(index=dupl_rows1)
df1 = df1.drop_duplicates(keep = 'first')
#display(df1[df1.Mileage==0]) # тоже вот очень странно(машинам по 10 и более лет некоторым, а у них 0 пробег)
# я думаю, что это тоже ошибки в данных, поэтому удаляю
ii1 = df1[df1.Mileage==0].index
df1 = df1.drop(index=ii1)
# посмотрим как отмечал выше на 0 и 20л двигатель
#display(df1[df1['Engine_volume'] == 0])
#display(df1[df1['Engine_volume'] == 20])
# это тоже является некорректными данными, которые не соответствуют действительности
## Объем двигателя Тойота Аква составляет 1.5 л. Мощность двигателей Toyota Aqua от 74 до 91 л.
##Автомобиль комплектовался моторами объёмом: 1.6, 2.0, 2.4 л.(Соната)
#удаляем
i1 = df1[df1['Engine_volume'] == 0].index
df1 = df1.drop(index=i1)

iii1 = df1[df1['Engine_volume'] == 20].index
df1 = df1.drop(index=iii1)
df1 = df1.drop(columns=['Model'])
df1 = df1.drop(columns=['Doors','Cylinders','Leather_interior'])
display(df1.head(3))

In [ ]:
# я это расписывал в файле Block2 и features, что сделаю заранее преобразование столбцов (пробег и налоги)
# возьму преобразование Бокса-Кокса для пробег и налога
# преобраазуем переменные пробег и налог
powerr = PowerTransformer(method='box-cox') 
minmaxx = MinMaxScaler()

milee = powerr.fit_transform(minmaxx.fit_transform(pd.DataFrame(df1.Mileage)) + 1)
taxx = powerr.fit_transform(minmaxx.fit_transform(pd.DataFrame(df1.Tax)) + 1)


df1.Mileage = milee
df1.Tax = taxx
#преобразовываю данные и записываю в колонки моего Х после трансформера
# скорее всего это преобразование буду делать до трансформера,то есть сразу преобразовывать фрейм по этим колонкам
# тк трудно будет сделать преобразования после pipeline

In [ ]:
# делаем маску
mask = df1.index

In [ ]:
# Блок(и) обучения и поверки модели

In [ ]:
finish_pipe =  Pipeline([
            ('preprocessing', CT), 
            ('estimator',RandomForestRegressor(n_estimators=3500))
            ])

finish_pipe.fit(X, y)
y_pred = finish_pipe.predict(X)
print('R2',r2_score(y,y_pred))
print('MSE', mean_squared_error(y,y_pred))
print('MAPE',mean_absolute_percentage_error(y,y_pred))

In [ ]:
# Блок предсказания с использованием тестового набора

In [ ]:
y_predict = finish_pipe.predict(df1)
y_predict.shape

In [ ]:
# Маска отобраных номеров строк для предсказания из тестового набора
mask = mask

# Название вектора предсказанных значений  y_predict полученого на основании тестового набора
y_predict = y_predict
